In [ ]:
import tkinter as tk
from tkinter import messagebox, filedialog
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np
from matplotlib.patches import Patch
from matplotlib.font_manager import FontProperties
import pandas as pd  # Import pandas for CSV handling

class TrafficManagementApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Sustainable Urban Traffic Management")
        
        # Maximize the window but retain title bar
        self.master.state('zoomed')

        # Create a canvas to hold the gradient background
        self.bg_canvas = tk.Canvas(self.master)
        self.bg_canvas.pack(fill=tk.BOTH, expand=True)

        # Create a gradient background
        self.create_gradient_background()

        # Create a scrollable canvas
        self.canvas = tk.Canvas(self.bg_canvas, bg="#f0f0f0")
        self.scrollbar = tk.Scrollbar(self.bg_canvas, command=self.canvas.yview)
        self.scrollable_frame = tk.Frame(self.canvas, bg="#f0f0f0")

        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: self.canvas.configure(scrollregion=self.canvas.bbox("all"))
        )

        self.canvas.create_window((0, 0), window=self.scrollable_frame, anchor="center")
        self.canvas.configure(yscrollcommand=self.scrollbar.set)

        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        # Create input fields and graph container
        self.create_input_fields()
        self.graph_container = tk.Frame(self.scrollable_frame, bg="#f0f0f0")
        self.graph_container.grid(row=20, column=0, columnspan=2, sticky="nsew", pady=20)

        # Configure styles for entry widgets
        style = ttk.Style()
        style.configure("CustomEntry.TEntry", relief="flat", padding=5, font=('Verdana', 12))
        style.map("CustomEntry.TEntry", background=[('active', '#e0f7fa'), ('!focus', '#ffffff'), ('focus', '#e0f7fa')],
                  bordercolor=[('active', '#00796b'), ('!focus', '#009688'), ('focus', '#00796b')],
                  lightcolor=[('active', '#b2ebf2'), ('!focus', '#ffffff'), ('focus', '#b2ebf2')],
                  darkcolor=[('active', '#004d40'), ('!focus', '#ffffff'), ('focus', '#004d40')])

        # Add a tooltip for buttons
        self.tooltip = None

        # Load data from CSV file
        self.load_data_from_csv('C:\\Users\\bharg\\Downloads\\singapore_dataset.csv')

    def create_gradient_background(self):
        # Create a gradient background
        width = self.master.winfo_screenwidth()
        height = self.master.winfo_screenheight()

        # Create a gradient from light blue to light green
        for i in range(height):
            color = self.interpolate_color("#a3d8e0", "#c2f0c2", i / height)
            self.bg_canvas.create_line(0, i, width, i, fill=color)

    def interpolate_color(self, start_color, end_color, factor):
        # Interpolates between two hex colors
        r1, g1, b1 = self.hex_to_rgb(start_color)
        r2, g2, b2 = self.hex_to_rgb(end_color)
        r = int(r1 + (r2 - r1) * factor)
        g = int(g1 + (g2 - g1) * factor)
        b = int(b1 + (b2 - b1) * factor)
        return f'#{r:02x}{g:02x}{b:02x}'

    def hex_to_rgb(self, hex_color):
        # Convert hex to RGB
        hex_color = hex_color.lstrip('#')
        return tuple(int(hex_color[i:i + 2], 16) for i in (0, 2, 4))

    def create_input_fields(self):
        # Create input fields with modern design and unique styles
        self.inputs = {
            "PM2.5": tk.StringVar(),
            "PM10": tk.StringVar(),
            "CO": tk.StringVar(),
            "NO2": tk.StringVar(),
            "CO2": tk.StringVar(),
            "CH4": tk.StringVar(),
            "N2O": tk.StringVar(),
            "Fuel Usage (L)": tk.StringVar(),
            "Route Efficiency (%)": tk.StringVar(),
            "Passengers (daily)": tk.StringVar(),
            "Food Waste (tons)": tk.StringVar(),
            "Urban Agriculture (%)": tk.StringVar(),
            "Food Distribution Efficiency (%)": tk.StringVar(),
            "EV Stations": tk.StringVar(),
            "Charging Efficiency (%)": tk.StringVar(),
            "EV Adoption Rate (%)": tk.StringVar()
        }

        row = 0
        for label in self.inputs.keys():
            tk.Label(self.scrollable_frame, text=label, bg="#f0f0f0", font=('Verdana', 12)).grid(row=row, column=0, padx=10, pady=10, sticky="e")
            
            # Styled Entry
            entry = ttk.Entry(self.scrollable_frame, textvariable=self.inputs[label], style='CustomEntry.TEntry')
            entry.grid(row=row, column=1, padx=10, pady=10, sticky="w")
            row += 1

        # Polished button with hover effect
        submit_button = tk.Button(self.scrollable_frame, text="Display Graphs", command=self.display_all_graphs,
                                  font=('Verdana', 12, 'bold'), bg='#4CAF50', fg='white', bd=0, relief='flat')
        submit_button.grid(row=row, columnspan=2, pady=20)

        # Bind hover effect to the button
        submit_button.bind("<Enter>", lambda e: self.on_hover(submit_button, True))
        submit_button.bind("<Leave>", lambda e: self.on_hover(submit_button, False))

    def on_hover(self, button, hover):
        if hover:
            button['bg'] = '#388e3c'  # Darker green on hover
        else:
            button['bg'] = '#4CAF50'  # Original green

    def clear_graphs(self):
        for widget in self.graph_container.winfo_children():
            widget.destroy()

    def display_graphs(self, data, title, graph_type="bar", bg_color="#ffffff"):
        graph_frame = tk.Frame(self.graph_container, bg=bg_color, bd=2, relief="ridge")
        graph_frame.pack(fill=tk.BOTH, expand=True, pady=10)

        fig, ax = plt.subplots(figsize=(8, 6))
        fig.patch.set_facecolor('#f2f2f2')  

        labels, values = zip(*data.items())  
        custom_font = FontProperties(family='Verdana', weight='bold', size=12)

        if graph_type == "bar":
            colors = plt.cm.coolwarm(np.linspace(0, 1, len(values)))
            bars = ax.bar(labels, values, color=colors, edgecolor='black')

            for bar in bars:
                yval = bar.get_height()
                ax.text(bar.get_x() + bar.get_width() / 2, yval + 0.1, round(yval, 2),
                        ha='center', va='bottom', fontsize=10, fontweight='bold')

            ax.set_xlabel('Parameters', fontsize=14, fontweight='bold', color='#333333')
            ax.set_ylabel('Values', fontsize=14, fontweight='bold', color='#333333')
            ax.set_title(title, fontsize=16, fontweight='bold', color='#333333')

        elif graph_type == "pie":
            wedges, texts, autotexts = ax.pie(values, labels=labels, autopct='%1.1f%%', startangle=90,
                                              colors=plt.cm.Set3(np.linspace(0, 1, len(values))),
                                              wedgeprops={'edgecolor': 'black'})

            for text in texts + autotexts:
                text.set_fontproperties(custom_font)
                text.set_color('#2d2d2d')

            ax.axis('equal')  
            ax.set_title(title, fontsize=16, fontweight='bold', color='#333333')

            patches = [Patch(facecolor=wed.get_facecolor(), label=label) for wed, label in zip(wedges, labels)]
            ax.legend(handles=patches, loc='center left', bbox_to_anchor=(1, 0.5), frameon=False)

        plt.xticks(rotation=45, ha='right', fontsize=10, fontweight='bold')
        plt.tight_layout()

        canvas = FigureCanvasTkAgg(fig, master=graph_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

    def load_data_from_csv(self, filepath):
        try:
            # Load data from the specified CSV file
            self.data = pd.read_csv(filepath)

            # Check if the expected columns exist in the CSV
            expected_columns = [
                "PM2.5", "PM10", "CO", "NO2", "CO2", "CH4", "N2O", 
                "Fuel Usage (L)", "Route Efficiency (%)", "Passengers (daily)", 
                "Food Waste (tons)", "Urban Agriculture (%)", 
                "Food Distribution Efficiency (%)", "EV Stations", 
                "Charging Efficiency (%)", "EV Adoption Rate (%)"
            ]

            missing_columns = [col for col in expected_columns if col not in self.data.columns]
            if missing_columns:
                raise ValueError(f"Missing columns in CSV: {', '.join(missing_columns)}")

            # Initialize input fields with data from the first row of the CSV
            for col in expected_columns:
                self.inputs[col].set(self.data[col].iloc[0])

        except FileNotFoundError:
            messagebox.showerror("Error", "CSV file not found. Please check the file path.")
        except ValueError as e:
            messagebox.showerror("Error", str(e))
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

    def display_all_graphs(self):
        self.clear_graphs()  # Clear previous graphs

        # Create a dictionary to hold the data for graphs
        graph_data = {
            "Air Quality": {
                "PM2.5": float(self.inputs["PM2.5"].get()),
                "PM10": float(self.inputs["PM10"].get()),
                "CO": float(self.inputs["CO"].get()),
                "NO2": float(self.inputs["NO2"].get()),
                "CO2": float(self.inputs["CO2"].get()),
                "CH4": float(self.inputs["CH4"].get()),
                "N2O": float(self.inputs["N2O"].get())
            },
            "Fuel Efficiency": {
                "Fuel Usage (L)": float(self.inputs["Fuel Usage (L)"].get()),
                "Route Efficiency (%)": float(self.inputs["Route Efficiency (%)"].get())
            },
            "Public Transport": {
                "Passengers (daily)": float(self.inputs["Passengers (daily)"].get()),
                "Food Waste (tons)": float(self.inputs["Food Waste (tons)"].get())
            },
            "Sustainable Practices": {
                "Urban Agriculture (%)": float(self.inputs["Urban Agriculture (%)"].get()),
                "Food Distribution Efficiency (%)": float(self.inputs["Food Distribution Efficiency (%)"].get())
            },
            "Electric Vehicles": {
                "EV Stations": float(self.inputs["EV Stations"].get()),
                "Charging Efficiency (%)": float(self.inputs["Charging Efficiency (%)"].get()),
                "EV Adoption Rate (%)": float(self.inputs["EV Adoption Rate (%)"].get())
            }
        }

        # Display the graphs for each category
        for title, data in graph_data.items():
            self.display_graphs(data, title)

# Create the main application window
if __name__ == "__main__":
    root = tk.Tk()
    app = TrafficManagementApp(root)
    root.mainloop()